In [1]:
import GasParsing as gp
import importlib

In [3]:
network_file = 'GasLib-11/GasLib_11.net'
scenario_file = 'GasLib-11/GasLib_11.scn'
ampl_dat_file = 'GasLib-11/GasLib-11.dat'

In [4]:
importlib.reload(gp)
network = gp.get_objects(network_file, scenario_file)

no dataclass: valve


In [5]:
lines = ['data;']

lines.append('\n### SETS ###\n')

# NODES
lines.append('set NODES :=')
for data_type, name in network:
    if data_type=='source':
        lines.append(name)
for data_type, name in network:
    if data_type=='sink':
        lines.append(name)
for data_type, name in network:
    if data_type=='innode':
        lines.append(name)
lines.append(';')    

# SOURCES
lines.append('\nset SOURCES :=')
for data_type, name in network:
    if data_type=='source':
        lines.append(name)
lines.append(';')    

# SINKS
lines.append('\nset SINKS :=')
for data_type, name in network:
    if data_type=='sink':
        lines.append(name)
lines.append(';')

In [58]:
# ARCS
lines.append('\nset ARCS :=')
for data_type, name in network:
    if data_type=='pipe':
        obj = network[(data_type, name)]
        in_node = obj._from
        out_node = obj.to
        temp = f'({in_node}, {out_node})'
        print(temp)
        lines.append(temp)
        
for data_type, name in network:
    if data_type=='compressorStation':
        obj = network[(data_type, name)]
        in_node = obj._from
        out_node = obj.to
        temp = f'({in_node} , {out_node})'
        print(temp)
        lines.append(temp)
lines.append(';')    

# PIPES
lines.append('\nset PIPES :=')
for data_type, name in network:
    if data_type=='pipe':
        obj = network[(data_type, name)]
        in_node = obj._from
        out_node = obj.to
        temp = f'({in_node} , {out_node})'
        print(temp)
        lines.append(temp)
lines.append(';')    

# COMPRESSIONSTATIONS
lines.append('\nset  COMPRESSORS:=')
for data_type, name in network:
    if data_type=='compressorStation':
        obj = network[(data_type, name)]
        in_node = obj._from
        out_node = obj.to
        temp = f'({in_node},{out_node})'
        print(temp)
        lines.append(temp)
lines.append(';')

(N02, N04)
(N01, N02)
(entry01, entry03)
(N02, exit01)
(N05, exit02)
(N05, exit03)
(entry02, N03)
(N03, N04)
(entry03 , N01)
(N04 , N05)
(N02 , N04)
(N01 , N02)
(entry01 , entry03)
(N02 , exit01)
(N05 , exit02)
(N05 , exit03)
(entry02 , N03)
(N03 , N04)
(entry03,N01)
(N04,N05)


In [59]:
lines.append('\n### PARAMS ###\n')

lines.append('# for NODES')
lines.append('param: FlowInOut FlowLowerNode FlowUpperNode PressureLower PressureUpper :=')

for data_type, name in network:
    if data_type=='source':
        obj = network[(data_type, name)]
        flow = obj.get_flow()
        lines.append(f'{name} {flow} {obj.flowMin} {obj.flowMax} {obj.pressureMin} {obj.pressureMax}')
        #print(f'{data_type}, {name}, {flowinout}')
        
for data_type, name in network:        
    if data_type == 'sink':
        obj = network[(data_type, name)]
        flow = obj.get_flow()
        lines.append(f'{name} {flow} {obj.flowMin} {obj.flowMax} {obj.pressureMin} {obj.pressureMax}')
        
for data_type, name in network:        
        # Innode doesn't have flow, flowMin and flowMax so manually input value -inf (put it in class)
    if data_type == 'innode':
        obj = network[(data_type, name)]
        flow = obj.get_flow()
        lines.append(f'{name} {flow} {obj.flowMin} {obj.flowMax} {obj.pressureMin} {obj.pressureMax}')
lines.append(';')   

lines.append('# for SOURCES')
lines.append('\nparam: CalorificValue:=')

# we do not have upper & lower bounds for source in .net files - used min and max of Hc[u]
for data_type, name in network:
    if data_type == 'source':
        obj = network[(data_type, name)]
        lines.append(f'{name} {obj.calorificValue}')
lines.append(';')                

lines.append('# for SINKS')
lines.append('\nparam: FlowLowerArc FlowUpperArc :=')

for data_type, name in network:
    if data_type == 'pipe':
        obj = network[(data_type, name)]
        in_node = obj._from
        out_node = obj.to
        temp = f'{in_node} {out_node}'
        lines.append(f'{temp} {obj.flowMin} {obj.flowMax}')
        
for data_type, name in network:        
    if data_type == 'compressorStation':
        obj = network[(data_type, name)]
        in_node = obj._from
        out_node = obj.to
        temp = f'{in_node} {out_node}'
        lines.append(f'{temp} {obj.flowMin} {obj.flowMax}')
lines.append(';')   
        
# This value is not given; Further review needed
# lines.append('param: MeanTemp :=')
# for data_type, name in network:
#     if data_type == 'pipe':
#         obj = network[(data_type, name)]
#         lines.append(f'{name} {}')


lines.append('# for PIPES')
lines.append('\nparam: Roughness Diameter Length :=')

for data_type, name in network:
    if data_type == 'pipe':
        obj = network[(data_type, name)]
        in_node = obj._from
        out_node = obj.to
        temp = f'{in_node} {out_node}'
        lines.append(f'{temp} {obj.roughness} {obj.diameter} {obj.length}')
        


lines.append('# for COMPRESSORSTATIONS')
lines.append('\nparam:  PressureChangeUpper :=')

for data_type, name in network:
    if data_type == 'compressorStation':
        obj = network[(data_type, name)]
        in_node = obj._from
        out_node = obj.to
        temp = f'{in_node} {out_node}'
        lines.append(f'{temp} {obj.pressureOutMax - obj.pressureInMin}')
        
for data_type, name in network:        
    if data_type == 'controlValve':
        obj = network[(data_type, name)]
        in_node = obj._from
        out_node = obj.to
        temp = f'{in_node} {out_node}'
        lines.append(f'{temp} {obj.pressureOutMax - obj.pressureInMin}')
lines.append(';')   

In [60]:
print(lines)

['data;', '\n### SETS ###\n', 'set NODES :=', 'entry02', 'entry03', 'entry01', 'exit03', 'exit02', 'exit01', 'N02', 'N03', 'N05', 'N04', 'N01', ';', '\nset SOURCES :=', 'entry02', 'entry03', 'entry01', ';', '\nset SINKS :=', 'exit03', 'exit02', 'exit01', ';', '\nset ARCS :=', '(N02, N04)', '(N01, N02)', '(entry01, entry03)', '(N02, exit01)', '(N05, exit02)', '(N05, exit03)', '(entry02, N03)', '(N03, N04)', '(entry03 , N01)', '(N04 , N05)', ';', '\nset PIPES :=', '(N02 , N04)', '(N01 , N02)', '(entry01 , entry03)', '(N02 , exit01)', '(N05 , exit02)', '(N05 , exit03)', '(entry02 , N03)', '(N03 , N04)', ';', '\nset  COMPRESSORS:=', '(entry03,N01)', '(N04,N05)', ';', '\n### PARAMS ###\n', 'param: FlowInOut FlowLowerNode FlowUpperNode PressureLower PressureUpper :=', 'exit03 80.00 40.0 600.0 40.0 60.0', 'N02 0 0 0 40.0 70.0', 'exit02 120.00 80.0 400.0 40.0 60.0', 'N03 0 0 0 40.0 70.0', 'N05 0 0 0 40.0 70.0', 'N04 0 0 0 40.0 70.0', 'entry02 140.00 100.0 500.0 40.0 70.0', 'exit01 100.00 50.0 

In [61]:
with open(ampl_dat_file, 'w') as f:
    f.write('\n'.join(lines))